### Bu modül belli başlı hisselerin son 4 çeyrek mali tablolarını Financials_202209 isimli klasöre Excel formatında kaydediyor

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import xlrd
import os
from tqdm import tqdm

import openpyxl
from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter

import yfinance as yf
from datetime import date, datetime


In [2]:
# Bu hücre hisse listesini çekmek için kullanılıyor, buna gerek yok bir config.yaml içerisinde halledilebilir.


current_period = '2022/9'
examplary_stock = 'ARCLK' # malesef ilk önce bir hisseyi kendimiz çekmek durumundayız. Openpyxl kütüphanesi excel
                            # formüllerini otomatik olarak kaydıramıyor, text olarak algılıyor. 

# temel analiz modülünün bulunduğu directory. 
tam_directory = 'C:\\Users\\Gökhan Akay\\Desktop\\Google Drive\\TAM_202203.xlsx'

#workbook = xlrd.open_workbook(tam_directory)
workbook = openpyxl.load_workbook(tam_directory)
#worksheet = workbook.sheet_by_name('Main')
worksheet = workbook.worksheets[3]


stock_list = []
for i in range(1,100):
    stock_list.append(worksheet.cell(i, 25).value)

stock_list = stock_list[1:58]  

In [4]:
# halihazırda çektiğimiz hisseleri buluyoruz.
os.chdir('C:\\Users\\Gökhan Akay\Desktop\Masaüstü\GA-Notebooks\Stocks To DB\Financials_202209')
#C:\Users\Gökhan Akay\Desktop\Masaüstü\GA-Notebooks\Stocks To DB\Financials_202209
current_stocks = [file[:-5] for file in os.listdir() if file.endswith('.xlsx')]

In [5]:
# halihazırda çektiklerimizi eliyoruz. 
stock_list = sorted(list(set(stock_list).difference(current_stocks)))

In [6]:
latest_quarters = []
print("Latest quarters are being checked...")
for i in tqdm (range(0,len(stock_list)), desc="Loading…", ascii=False, ncols=75):
    stock = stock_list[i]
    html_text = requests.get('https://www.isyatirim.com.tr/tr-tr/analiz/hisse/Sayfalar/sirket-karti.aspx?hisse='+stock).text

    soup = BeautifulSoup(html_text, 'lxml')
    table = soup.find('tbody', id = 'tbodyMTablo')
    trs = table.find_all('tr')    

    quarter_list = []
    for i in range(0,4):
        quarter_list.append(soup.find_all('th', class_ = 'form-group')[0].find_all('option')[i].text)

    latest_quarters.append(quarter_list[0])

Latest quarters are being checked...


Loading…: 100%|████████████████████████████| 57/57 [04:07<00:00,  4.33s/it]


In [8]:
#burda son periyodu bizim yukarıda belirttiğimiz periyot olan hisseleri buluyoruz.
stock_period = pd.DataFrame(zip(stock_list,latest_quarters))
stock_period.columns = ['stock','latest_period']
stock_period.set_index('stock', inplace = True)
stocks_to_pull = list(stock_period[stock_period.latest_period == current_period].index)    
if len(stocks_to_pull) == 0:
    print("There is no stocks to pull. Program is terminating.")
else:
    print("These stocks will be pulled:", stocks_to_pull)

These stocks will be pulled: ['AEFES', 'AKSA', 'AKSEN', 'ALARK', 'ALKA', 'ALKIM', 'ARCLK', 'ARDYZ', 'ASELS', 'AYGAZ', 'BIMAS', 'BRISA', 'CCOLA', 'CEMTS', 'CIMSA', 'DEVA', 'DOAS', 'ECILC', 'EGEEN', 'ENJSA', 'ENKAI', 'ERBOS', 'EREGL', 'FROTO', 'GEDZA', 'GOODY', 'HEKTS', 'JANTS', 'KCHOL', 'KORDS', 'KOZAL', 'KRDMD', 'LOGO', 'MGROS', 'ORGE', 'OTKAR', 'OYAKC', 'PETKM', 'PGSUS', 'SAHOL', 'SARKY', 'SASA', 'SISE', 'SOKM', 'TCELL', 'THYAO', 'TKFEN', 'TMSN', 'TOASO', 'TTKOM', 'TTRAK', 'TUPRS', 'ULKER', 'ULUSE', 'VESBE', 'VESTL', 'YATAS']


In [9]:
# çekilecek her bir hisse için döngüye giriyoruz.
for i in range(0,len(stocks_to_pull)):
    
    stock = stocks_to_pull[i]
    html_text = requests.get('https://www.isyatirim.com.tr/tr-tr/analiz/hisse/Sayfalar/sirket-karti.aspx?hisse='+stock).text

    soup = BeautifulSoup(html_text, 'lxml')
    table = soup.find('tbody', id = 'tbodyMTablo')
    trs = table.find_all('tr')
    
    quarter_list = []
    for i in range(0,4):
        quarter_list.append(soup.find_all('th', class_ = 'form-group')[0].find_all('option')[i].text)
    
    val_list = []
    for i in range(1,len(trs)):
        val_list.append(trs[i].find_all('td'))
        
        
    col_list = []
    for i in range(0,len(val_list)):
        col_list.append(val_list[i][0].text)
        
    col_list[0]
    col_list.remove('Gelir Tablosu')
    try:
        col_list.remove('Dipnot')
    except:
        pass
    
    
    try:
        col_list.remove('Nakit Akım Tablosu')
    except:
        pass
    
    
        
    q1 = []    
    for i in range(0,len(val_list)):
        try:
            q1.append(val_list[i][1].text)
        except:
            i = i+1
            
    q1[0]
    
    q2 = []    
    for i in range(0,len(val_list)):
        try:
            q2.append(val_list[i][2].text)
        except:
            i = i+1
    
    q2[0]
    
    
    q3 = []    
    for i in range(0,len(val_list)):
        try:
            q3.append(val_list[i][3].text)
        except:
            i = i+1
    
    q3[0]
    
    q4= []    
    for i in range(0,len(val_list)):
        try:
            q4.append(val_list[i][4].text)
        except:
            i = i+1   
        
    q4[0]
    
    q1 = [w.replace('.', '') for w in q1]
    q1 = list(map(float, q1))

    q2 = [w.replace('.', '') for w in q2]
    q2 = list(map(float, q2))

    q3 = [w.replace('.', '') for w in q3]
    q3 = list(map(float, q3))

    q4 = [w.replace('.', '') for w in q4]
    q4 = list(map(float, q4))
    
    col_list.insert(0,'')
    q1.insert(0,'')
    q2.insert(0,'')
    q3.insert(0,'')
    q4.insert(0,'')
    
    col_list.insert(72,'')
    q1.insert(72,'')
    q2.insert(72,'')
    q3.insert(72,'')
    q4.insert(72,'')
    
    col_list.insert(115,'')
    q1.insert(115,'')
    q2.insert(115,'')
    q3.insert(115,'')
    q4.insert(115,'')
    
    
    df = pd.DataFrame(index = col_list)
    df['q1'] = q1
    df['q2'] = q2
    df['q3'] = q3
    df['q4'] = q4
    
    df.columns = quarter_list
    
    # gelir tablosu ve dipnot'tan sonra gelen kısmı eliyoruz. 
    df = df.iloc[0:124,:]
    
    #excel'e yazdırıyoruz. 
    df.to_excel(stock+'.xlsx')
    
    print(stock+' is pulled')


AEFES is pulled
AKSA is pulled
AKSEN is pulled
ALARK is pulled
ALKA is pulled
ALKIM is pulled
ARCLK is pulled
ARDYZ is pulled
ASELS is pulled
AYGAZ is pulled
BIMAS is pulled
BRISA is pulled
CCOLA is pulled
CEMTS is pulled
CIMSA is pulled
DEVA is pulled
DOAS is pulled
ECILC is pulled
EGEEN is pulled
ENJSA is pulled
ENKAI is pulled
ERBOS is pulled
EREGL is pulled
FROTO is pulled
GEDZA is pulled
GOODY is pulled
HEKTS is pulled
JANTS is pulled
KCHOL is pulled
KORDS is pulled
KOZAL is pulled
KRDMD is pulled
LOGO is pulled
MGROS is pulled
ORGE is pulled
OTKAR is pulled
OYAKC is pulled
PETKM is pulled
PGSUS is pulled
SAHOL is pulled
SARKY is pulled
SASA is pulled
SISE is pulled
SOKM is pulled
TCELL is pulled
THYAO is pulled
TKFEN is pulled
TMSN is pulled
TOASO is pulled
TTKOM is pulled
TTRAK is pulled
TUPRS is pulled
ULKER is pulled
ULUSE is pulled
VESBE is pulled
VESTL is pulled
YATAS is pulled
